In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import sncosmo
from   astropy.table    import Table
from   astropy.io       import ascii

from   analyzeSN import LightCurve
import analyzeSN

import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as mb

#### This file can be replaced with any of the opsim files

In [2]:
opsfilename = 'minion_1016_newsky.db'
opsdb       = db.OpsimDatabase(opsfilename)

simdata = opsdb.query_columns(opsdb.defaultTable, 
                              colnames=['fieldRA', 'fieldDec', 'filter', 'night', 'visitTime', 'airmass', 'filtSkyBrightness', 'expMJD', 'fiveSigmaDepth'], 
                              sqlconstraint=None)

### Putting OpSim columns in arrays to make it easier to work with

In [3]:
ra_array      = np.zeros(len(simdata))
dec_array     = np.zeros(len(simdata))
airmass_array = np.zeros(len(simdata))
time_array    = np.zeros(len(simdata))
sigmadepth    = np.zeros(len(simdata))

# creating empty array for strings for filter names
val           = [''] * len(simdata)
filter_array  = np.array(val)

for i in range(len(simdata)):
    ra_array[i]      = simdata[i][0] # right ascension
    dec_array[i]     = simdata[i][1] # declination
    filter_array[i]  = simdata[i][2] # filters
    time_array[i]    = simdata[i][7] # time should be expMJD
    airmass_array[i] = simdata[i][5] # airmass
    sigmadepth[i]    = simdata[i][8] # 5 sigma depth

#### Setting the filename of sampled supernova parameters. 
This file is obtained by running SNTable notebook that Rahul has. It has (x0,x1,c,t0,z) and host galaxy information. t0 is the peak magnitude time of each supernova.

In [5]:
#filename = "/global/homes/a/anitab/DC2_run1p1_TransientDocs/Notebooks/Hosted_SN_table.csv"
filename = "/global/homes/a/anitab/DC2_run1p1_TransientDocs/Notebooks/Hosted_SN_table_cosmoDC2_test.csv"
#filename = "/global/homes/a/anitab/DC2_run1p1_TransientDocs/Notebooks/Hosted_SN_table_cosmoDC2.csv"

from pandas import read_csv

data  = read_csv(filename)#, delim_whitespace=True)
#data = vvv['galaxy_id c mB t0 x0 x1 z random_Hosting rand_host zbin "Unnamed: 0" diskMassStellar morphology/diskHalfLightRadiusArcsec morphology/diskMinorAxisArcsec morphology/positionAngle morphology/spheroidHalfLightRadiusArcsec morphology/spheroidMinorAxisArcsec size_bulge_true size_disk_true totalMassStellar uniqueId raJ2000_gal decJ2000_gal z zbin_gals snid']

# uncomment this to see what columns are available
###vvv.columns 

data.set_index('snid', inplace=True)
snids = data.index.values

### Sampled SN properties in an array
$m_B = 10.635 - \log_{10}(x0)$

In [ ]:
'''
zvals   = data.z.values
x0vals  = data.x0.values
x1vals  = data.x1.values
cvals   = data.c.values
mBvals  = data.mB.values
t0vals  = data.t0.values
smass   = data.totalMassStellar.values
ravals  = data.raJ2000_gal.values
decvals = data.decJ2000_gal.values

snids = data.index.values
'''

In [8]:
z_condition = 'z < 0.1'

zvals   = data.loc[snids].query(z_condition).z.values
x0vals  = data.loc[snids].query(z_condition).x0.values
x1vals  = data.loc[snids].query(z_condition).x1.values
cvals   = data.loc[snids].query(z_condition).c.values
mBvals  = data.loc[snids].query(z_condition).mB.values
t0vals  = data.loc[snids].query(z_condition).t0.values
smass   = data.loc[snids].query(z_condition).totalMassStellar.values
ravals  = data.loc[snids].query(z_condition).ra.values
decvals = data.loc[snids].query(z_condition).dec.values

In [9]:
snids = data.loc[snids].query('z < 0.1').index.values

### Choosing rows of OpSim based on sampled SNe

In [11]:
import os
import opsimsummary as oss
import healpy as hp
print(oss.__version__)

Some imports failed, which implies some dependencies are missing as described below
No module named 'mpl_toolkits.basemap'
Visulization functions based on maps will not work
1.16.1


In [12]:
from opsimsummary import SynOpSim

I need to make sure angleUnit has 'degrees' attribute

In [13]:
synopsim  = SynOpSim.fromOpSimDB(opsfilename, opsimversion='lsstv3',
                                angleUnit='degrees', usePointingTree=True)

 reading from database sqlite:///minion_1016_newsky.db
SELECT * FROM Summary WHERE propID in (56, 54)


### Generator
This is a Python generator function to make going through data faster. This goes through the observing file and uses Ra and Dec of sampled SNe to determine which SNe would be visible at which times.

In [14]:
gen = synopsim.pointingsEnclosing(ravals, decvals, circRadius=0., pointingRadius=1.75, usePointingTree=True)

### Time and filters from OpSim file
This is going through the generator and inserting times of observation and filer names in a list. After running this for loop, if we need to go through the generator again, we need to run the gen = ... line again because the generator has now gone all the way to the end. So, we need ro rerun this line:

gen = synopsim.pointingsEnclosing(ravals, decvals, circRadius=0., pointingRadius=1.75, usePointingTree=True)

In [15]:
timeopsim_list       = []
filteropsim_list     = []
sigmadepthopsim_list = []

for i, df in enumerate(gen):
    
    timeopsim = df['expMJD']
    timeopsim_list.append(timeopsim.values)
    
    filteropsim = df['filter']
    filteropsim_list.append(filteropsim.values)
    
    sigmadepthopsim = df['fiveSigmaDepth']
    sigmadepthopsim_list.append(sigmadepthopsim.values)
    
    #raopsim = df['fieldRA']
    #raopsim_list.append(raopsim.values)
    ###raopsim_list += raopsim.values.tolist() # this is to make the 2D array into 1D array. Not needed in general though

### Plotting SNe and their corresponding OpSim points
This shows supernovae as red dots and the OpSim visits as blue dots. There might be multiple visits at the same location so some points are overlapping and they are for different times.

In [ ]:
'''
ifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)
ify      = np.array([1,6,11,16,21])   # subplot numbers (for turning y ticks on/off)

plt.figure(1, figsize=(15,15))

for i in range(25):
    
    plt.subplot(5,5,i+1)
    plt.plot(np.rad2deg(decopsim_list[1000*i]), np.rad2deg(raopsim_list[1000*i]), ".", color="blue")
    plt.plot(decvals[1000*i], ravals[1000*i], ".", color="red")
    plt.ylim(50,60)
    plt.xlim(-34,-24)
    plt.tick_params(axis='both', labelleft='off', labelbottom='off')

    if i+1 in ifx:
        plt.xlabel("Ra [deg]",fontsize=15)
        plt.tick_params(axis='x', labelbottom='on')
    if i+1 in ify:
        plt.ylabel("Dec [deg]",fontsize=15)
        plt.tick_params(axis='y', labelleft='on')
    
plt.suptitle("SNe in red with opsim visits in blue")
'''

### Checking time range of OpSim with $t_0$
Looping over all supernova and checking the observing times corresponding to supernova location from OpSim file and adding the ones that are within (t0 - 19 < OpSimtime < t0+49 ) range. This is the SALT model time range. If the OpSim time is not within this time range, the flux calculated would be 0 and we do not get any useful information. So, I want to reduce the number of data and only work with the useful data.

In [16]:
from lsst.sims.catUtils.supernovae import SNObject
from lsst.sims.photUtils.BandpassDict import BandpassDict

# Setting the LSST BandPass object to be used in BandFlux function
LSST_BandPass = BandpassDict.loadTotalBandpassesFromFiles()

mlist     = []
jlist_tot = []
snids_tot = []

for m in range(len(timeopsim_list)):
    jlist = []
    for j in range(len(timeopsim_list[m])):
        if (timeopsim_list[m][j] > t0vals[m] -19 and timeopsim_list[m][j] < t0vals[m] + 49):
            jlist.append(j)
    if jlist:
        mlist.append(m)
        jlist_tot.append(jlist)
        

In [17]:
NumSN_query = np.shape(mlist)

(32,)

### Calculating the flux
Goes through a loop for each supernova and sets the parameters for each and then calculates the flux for all the visits

In [18]:
fluxlist_tot    = [] 
timelist_tot    = []
fluxerrlist_tot = []
filterlist_tot  = []

# Going through each SN
for SN_id, val in enumerate(mlist): #[:400]):
    # setting parameters for each supernova
    params   = {'x0': x0vals[val], 'x1': x1vals[val], 'c': cvals[val], 't0':t0vals[val], 'z':zvals[val]}
    snobject = SNObject(ravals[val], decvals[val])
    snobject.set(**params)
    #snobject.set_MWebv(0.0) # Finally we should not have this
    
    # to make filternames what the code accepts using list comprehension to make it faster
    filters_modified = [LSST_BandPass[key] for key in filteropsim_list[val]]

    fluxlist    = []
    timelist    = []
    fluxerrlist = []
    filterlist  = []

    
    # going through all the visits for each single supernova
    for j in jlist_tot[SN_id]:
        # flux and fluxerr
        fluxval    = snobject.catsimBandFlux(bandpassobject = filters_modified[j], time = timeopsim_list[val][j])
        fluxvalerr = snobject.catsimBandFluxError(bandpassobject=filters_modified[j],
                                                  time=timeopsim_list[val][j],
                                                  m5=sigmadepthopsim_list[val][j])
        
        # Adding error to flux values by sampling from a Gaussian with mean of fluxval 
        # and sigma of fluxvalerr for each point 
        fluxval_new = np.random.normal(fluxval, fluxvalerr, 1)[0] # this [0] is to avoid destorying the shape of list
        fluxlist.append(fluxval_new)
        
        timelist.append(timeopsim_list[val][j]) # appending the times of observation to a list
        
        fluxerrlist.append(fluxvalerr)
        filterlist.append(filteropsim_list[val][j])
    
    fluxlist_tot.append(fluxlist)       # shape should be (number of SN, entries for each SN)
    timelist_tot.append(timelist)       # shape should be (number of SN, entries for each SN)
    fluxerrlist_tot.append(fluxerrlist) # shape should be (number of SN, entries for each SN)
    filterlist_tot.append(filterlist)   # shape should be (number of SN, entries for each SN)

### Plotting flux vs. time
This is the fluxes obtained using catsimBandFlux() function for a selection of SNe vs expMJD time. The points are fluxes in all bands so that is why they have funny and oscillating shapes.

In [ ]:
ifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)
ify      = np.array([1,6,11,16, 21])

plt.figure(1, figsize=(15,15))

for i in range(25):
    plt.subplot(5,5,i+1)
    
    indx = np.arange(0,len(timelist_tot[i]))
    Z = [x for _,x in sorted(zip(timelist_tot[i],indx))]
        
    plt.errorbar(np.sort(timelist_tot[i]), np.array(fluxlist_tot[i])[Z], yerr = np.array(fluxerrlist_tot[i])[Z], color='blue')

    if i+1 in ifx:
        plt.xlabel("Time [MJD]",fontsize=15)
    #    plt.tick_params(axis='x', labelbottom='on')
    if i+1 in ify:
        plt.ylabel("Flux",fontsize=15)
    #    plt.tick_params(axis='y', labelleft='on')

### sncosmo fitting
https://sncosmo.readthedocs.io/en/v1.6.x/photdata.html?highlight=Photometric%20Data

I want to make a table that has all the information to pass to sncosmo. It should be a table that includes:
(time, band, flux, fluxerr, zp, zpsys). 

The band should be in the form of 'lsstu', 'lsstg', ...

zp: 'ab'

### Creating a table to pass to sncosmo
This function creates a table that has (time, band, flux, fluxerr, zp, zpsys) values in a Table that can be passed to sncosmo fit_lc() function. The format of the bands should be correct as it is done here.

In [20]:
from astropy.table import Table, Column

dust             = sncosmo.CCM89Dust()
modelsncosmo     = sncosmo.Model(source = 'salt2-extended', effects=[dust], effect_names=['mw'],
                                 effect_frames=['obs']) # setting SN model including extinction

def create_Table(SnNum, timelist=timelist_tot):
    t            = Table()
    
    length       = len(timelist[SnNum])  # length of time and flux array for each SN
    zpsysarray   = np.array(['ab'] * length) # creating an array of zpsys

    t['time']    = timelist_tot[SnNum]       # setting time
    
    # This line is to make bands in a format readable for sncosmo
    t['band']    = np.array(['lsst' + np.array(filterlist_tot[SnNum])[i] for i in range(len(filterlist_tot[SnNum]))])
    t['flux']    = fluxlist_tot[SnNum]       # flux
    t['fluxerr'] = fluxerrlist_tot[SnNum]    # fluxerr
    t['zp']      = np.zeros(length)          # zp (all zp values are 0)
    t['zpsys']   = zpsysarray                # zpsys

    t['band']    = t['band'].astype(str)     # to make sure band has the correct format
    
    return t

In [ ]:
#lc = LightCurve(t.to_pandas().query('band != "sdssy"').sort_values(by='time'))

In [24]:
t  = create_Table(16)
snids[mlist[16]]

modelsncosmo.set(z=data.loc[snids[mlist[16]]].z)
modelsncosmo.set(t0=data.loc[snids[mlist[16]]].t0)
result, fitted_model  = sncosmo.fit_lc(np.sort(t), modelsncosmo, ['x0', 'x1', 'c'])

In [26]:
params = Odict(data.loc[snids[mlist[16]]][['z', 'c', 'x1', 'x0', 't0']])

truth  = sncosmo.Model(source='salt2-extended')
truth.set(**params)

truth_salt = sncosmo.Model(source='salt2')
truth_salt.set(**params)

In [40]:
fitted_model.param_names

['z', 't0', 'x0', 'x1', 'c', 'mwebv', 'mwr_v']

### Plotting fitted SN fluxes

In [ ]:
fig = sncosmo.plot_lc(np.sort(t), model=(fitted_model, truth), errors=result.errors, 
                    model_label=("fit", "truth"), pull=True, color='k')

### $m_B$ and $\mu(z)$ functions

In [27]:
def mB_func(x0):
    return 10.635 - (2.5 * np.log10(x0))

def mu(x0, x1, c, smassval):
    
    alpha  = 0.141
    beta   = 3.101
    mB     = mB_func(x0)
    DeltaM = -0.07
    
    if smassval < 1e10:
        MB = -19.05
    else:
        MB = -19.05 + DeltaM
        
    val   = mB - (MB - (alpha * x1) + (beta * c))
    
    return val

### Error on $\mu(z)$

In [29]:
def sigma_mB(x0, sigma_x0):
    final_val = (((-2.5/np.log(10)) * (1./x0))**2) *  (sigma_x0**2)
    return np.sqrt(final_val)

def sigma_mu(sigma_mBval, sigma_x1, sigma_c):
    alpha     = 0.141
    beta      = 3.101
    val_err   = (sigma_mBval**2) + ((alpha**2) * (sigma_x1**2)) + ((beta**2) * (sigma_c**2)) 
    final_val = np.sqrt(val_err)
    return final_val

### Calculating $\mu$ for a subset of SNe

In [2]:
sigma_mu_vals = np.zeros(32)
mu_vals = np.zeros(32)
zvals_fitsn = np.zeros(32)
zvals_fitfit = np.zeros(32)

for i in range(32):
    
    print (i)
    modelsncosmo.set(z=data.loc[snids[mlist[i]]].z)
    modelsncosmo.set(t0=data.loc[snids[mlist[i]]].t0)
    result, fitted_model  = sncosmo.fit_lc(np.sort(t), modelsncosmo, ['x0', 'x1', 'c'])
    
    mu_vals[i] =  mu(fitted_model.parameters[2], fitted_model.parameters[3], fitted_model.parameters[4], 
        data.loc[snids[mlist[i]]].totalMassStellar)
    sigma_mBval = sigma_mB(fitted_model.parameters[2], result.errors['x0'])
    sigma_mu_vals[i] = sigma_mu(sigma_mBval,result.errors['x1'], result.errors['c'])
    
    zvals_fitsn[i] = fitted_model.parameters[0]
    zvals_fitfit[i] = snobject.SNstate['z'] 

"\nsigma_mu_vals = np.zeros(32)\nmu_vals = np.zeros(32)\nzvals_fitsn = np.zeros(32)\nzvals_fitfit = np.zeros(32)\n\nfor i in range(32):\n    \n    print (i)\n    modelsncosmo.set(z=data.loc[snids[mlist[i]]].z)\n    modelsncosmo.set(t0=data.loc[snids[mlist[i]]].t0)\n    result, fitted_model  = sncosmo.fit_lc(np.sort(t), modelsncosmo, ['x0', 'x1', 'c'])\n    \n    mu_vals[i] =  mu(fitted_model.parameters[2], fitted_model.parameters[3], fitted_model.parameters[4], \n        data.loc[snids[mlist[i]]].totalMassStellar)\n    sigma_mBval = sigma_mB(fitted_model.parameters[2], result.errors['x0'])\n    sigma_mu_vals[i] = sigma_mu(sigma_mBval,result.errors['x1'], result.errors['c'])\n    \n    zvals_fitsn[i] = fitted_model.parameters[0]\n    zvals_fitfit[i] = snobject.SNstate['z'] \n"

### Plotting distribution of $\mu$ obtained from SN fitting

In [4]:
indx_good = np.where(mu_vals>35) # to avoid the weird ones that have mu ~ 12 ish

ztheory = np.linspace(zvals_fitfit[indx_good].min(), zvals_fitfit.max(),20)
mu_theory_vect = np.vectorize(mu_kosowsky)
mu_theory = mu_theory_vect(ztheory)

plt.hist(mu_vals[indx_good], bins = 20, histtype = "step", color="black")
plt.xlabel(r"$\mu (z)$", fontsize=15)
plt.ylabel("N", fontsize=15)

'indx_good = np.where(mu_vals>35) # to avoid the weird ones that have mu ~ 12 ish\n\nztheory = np.linspace(zvals_fitfit[indx_good].min(), zvals_fitfit.max(),20)\nmu_theory_vect = np.vectorize(mu_kosowsky)\nmu_theory = mu_theory_vect(ztheory)\n\nplt.hist(mu_vals[indx_good], bins = 20, histtype = "step", color="black")\nplt.xlabel(r"$\\mu (z)$", fontsize=15)\nplt.ylabel("N", fontsize=15)'

### $\mu(z)$ from theory

In [57]:
from   scipy              import interpolate, integrate

h  = 0.6726
H0 = h*100.

def chi_integrand(zprime):
    '''
    Parameters
    -------------------------------------------------------------------------------------------------
        zprime: redshift values
        
    Return
    -------------------------------------------------------------------------------------------------
        Integrand of integral part in luminosity distance in Equation (3) in Kosowsky 2011.
        d(zprime)/ E(zprime)
    '''

    omegam = 0.316
    omegav = 0.684
    val    = 1./(np.sqrt(omegam*(1+zprime)**3 + omegav))
    return val

def chi_integral(z):
    '''
    Parameters
    -------------------------------------------------------------------------------------------------
        z: redshift
        
    Return
    -------------------------------------------------------------------------------------------------
        Integral of the integral in luminsoty distance in Equation (3) in Kosowsky 2011.
        d(zprime)/ E(zprime) integrated from 0 to z
    '''
    
    c       = 3e5 #km/s
    val,err = integrate.quad(chi_integrand,0,z)
    return val

def mu_kosowsky(z):
    '''
    Parameters
    -------------------------------------------------------------------------------------------------
        z: redshift
        
    Return
    -------------------------------------------------------------------------------------------------
        Distance modulus (mu) obtained from Equation (2) in Kosowsky
    '''
    
    c   = 3e5
    righthandside = (1+z) * (c/H0) * chi_integral(z)
    val = 2.17 * np.log(righthandside) + 25.
    return val

### Number of SN available
To find which SNe have larger number of times so that we have more points to fit for the fluxes in each band

In [ ]:
for i in range(len(mlist)):
    if np.shape(timelist_tot[i])[0] > 500:
        print (i)
        
print ("Total number of SN = ", np.shape(timelist_tot[16]))